<a href="https://colab.research.google.com/github/mingmcs/cs598ccc/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install redis
!pip install openai==0.28
!pip install cohere
!pip install tiktoken
!pip install langchain
!pip install typing
!pip install scikit-learn
!git clone https://github.com/LogSummary/code-and-datasets.git

from PyPDF2 import PdfReader
#from openai.embeddings_utils import get_embedding
import tiktoken
from typing import List
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis as RedisVectorStore
from langchain.document_loaders import DataFrameLoader



In [ ]:
!pip show openai



In [ ]:
import json
import pandas as pd


proxifier_log_fie = "/content/code-and-datasets/data/summary/logs/Proxifier.txt"

SUMMARY = 'summary'
LOG = 'log'
def get_record_type(key):
  if key.startswith("#summary:#"):
    return SUMMARY
  else:
    return LOG

def load_data(file):
  data_log = {}
  data_summary = []
  current_key = None
  lines = []
  record_type = LOG

  with open(file, 'r') as file:
    for line in file:
      line = line.strip()
      if not line:
        continue

      if line.startswith("#"):
        if current_key is not None:
          if get_record_type(current_key) == SUMMARY:
            data_summary.append(lines)
          else:
            data_log[current_key.replace("#", "")] = lines

        current_key = line
        lines = []
      else:
        lines.append(line)

    if current_key is not None:
      data_summary.append(lines)
  return data_log, data_summary

p_data, p_summary = load_data(proxifier_log_fie)
#print(p_data['1'])

a = ""
for i in range(0, len(p_data['1'])):
  a = a + p_data['1'][i] + '\n '
print(a)

print(p_summary[0][0])

df = pd.DataFrame(columns=[ 'log_data', 'log_summary', 'product'])
for i in range(1, len(p_data)+1):
  #print(p_data[str(i)])
  #print(p_summary[i-1][0])
  string_data = ""
  for j in range(0, len(p_data[str(i)])):
    string_data = string_data + p_data[str(i)][j] + "\n "
  df = df.append({'log_data': string_data, 'log_summary':p_summary[i-1][0], 'product':"Proxifier"} , ignore_index=True)

print(df.items)

In [ ]:
zookeeper_log_fie = "/content/code-and-datasets/data/summary/logs/Zookeeper.txt"
z_data, z_summary = load_data(zookeeper_log_fie)
print(len(z_data))
print( ', '.join(z_data.keys()))
print('\n'.join(z_data['1']))
print(z_summary[0])
print(len(z_data['2']))
print(len(z_summary))


for i in range(1, len(z_data)+1):
  #print(p_data[str(i)])
  #print(p_summary[i-1][0])
  string_data = ""
  for j in range(0, len(z_data[str(i)])):
    string_data = string_data + z_data[str(i)][j] + "\n "
  df = df.append({'log_data': string_data, 'log_summary':z_summary[i-1][0], 'product':"Zookeeper"} , ignore_index=True)

print(df.items)



b = ""
for i in range(0, len(z_data['1'])):
  b = b + z_data['1'][i] + '\n '
print(b)



In [ ]:
!pip install redis

In [ ]:
# this is used to test pandas
import redis
r = redis.Redis(host='redislocal', port=6379, decode_responses=True)
r.set('foo', 'bar')
a = r.get('foo')
print(a)

# New Section

In [ ]:
API_KEY = "" # supply your api_key
RESOURCE_ENDPOINT = "https://openaics598.openai.azure.com"
DEPLOYMENT_NAME = "cs598"
MODEL_NAME = "text-embedding-ada-002"


In [ ]:
embedding = OpenAIEmbeddings(
    deployment=DEPLOYMENT_NAME,
    model=MODEL_NAME,
    openai_api_base=RESOURCE_ENDPOINT,
    openai_api_type="azure",
    openai_api_key=API_KEY,
    openai_api_version="2023-05-15",
    chunk_size=16 # current limit with Azure OpenAI service. This will likely increase in the future.
    )

In [ ]:
print(embedding)
loader = DataFrameLoader(df, page_content_column="log_data" )
movie_list = loader.load()
print(movie_list)
redis_url = "redis://redislocal:6379"

'''
vectorstore = RedisVectorStore.from_documents(
    documents=movie_list,
    embedding=embedding,
    index_name="page_num",
    redis_url=redis_url
)
'''
vectorstore = RedisVectorStore.from_documents(
    documents=movie_list,
    embedding=embedding,
    index_name="log_data",
    redis_url=redis_url
)

In [ ]:
"""
query = "Version 3 features a number of other improvements. Overall the user experience has been"
results = vectorstore.similarity_search_with_score(query, k=10)
for i, j in enumerate(results):
  #print(results[i])
  print(j)
"""
#query = "to match the rule a connection should connect to a host from this list."
#query = "- proxy.cse.cuhk.edu.hk : 5070 open through proxy proxy.cse.cuhk.edu.hk"


print(a)
results = vectorstore.similarity_search_with_score(a, k=5)
for i, j in enumerate(results):
  print(results[i][1])
  print(j)



In [ ]:

print(b)
results = vectorstore.similarity_search_with_score(b, k=5)
for i, j in enumerate(results):
  #print(results[i][0])
  #print(j[0].page_content)
  print(j[0].metadata.get('log_summary'))

In [ ]:
def open_ai_zk_summary(model, input, train):
  text = '\n'.join(input)
  # print(input)

  prompts=[]
  prompts.append({"role": "system", "content": "You are a zookeeper specialist"})
  prompts.append({"role": "user", "content": "Help me to create a concise summary less than 20 words, \
  the output is a json summary with the following keys: summary, anormly"})
  prompts.append({"role": "assistant", "content": "yes, I can do it."})

  logs, summary = train[0], train[1]

  """
  for i in range(len(train[0])):
    userContent = '\n'.join(logs[i])
    assistantContent = summary[i][0]
    prompts.append({"role": "user", "content": f"""create a summary for: {userContent}"""})
    # data = f""""summary": {assistantContent}"""
    prompts.append({"role": "assistant", "content": assistantContent})
  """
  results = vectorstore.similarity_search_with_score(text, k=5)
  for i, j in enumerate(results):
    userContent = j[0].page_content
    assistantContent = j[0].metadata.get('log_summary')
    prompts.append({"role": "user", "content": f"""create a summary for: {userContent}"""})
    prompts.append({"role": "assistant", "content": assistantContent})


  prompts.append({"role": "user", "content": f"create a summary for: {text}"})

  # for i in range(len(prompts)):
  #   print(prompts[i])

  response = client.chat.completions.create(
    model=model,
    messages=prompts,
    temperature=0.3,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
  )

  print(f'response:{response.choices[0].message.content}')

  return response.choices[0].message.content